In [3]:
import os
import math
import numpy as np
import pandas as pd
import ccxt
import talib
from datetime import datetime
import tensorflow as tf
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from script.fetch_history_data import binance_fetch_history_price, binance_single_fetch_history_price
from script.preprocess import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from script.transformer_timestep import *
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

Tensorflow version: 2.6.5


In [4]:
batch_size = 32
seq_len = 128
d_k = 256
d_v = 256
n_heads = 12
ff_dim = 256
valid_percent = 20
test_percent = 10

target_column='Close_MA_ptc_10'
pearson_corr_cutoff = 0.2

In [5]:
df_cal = pd.read_pickle('./data/multi_factor_v2.0.2.pickle')
df_train, df_val, df_test = preprocess_train_test_split(df_cal, valid_percent=valid_percent, test_percent=test_percent)

In [49]:
print(df_train.shape, df_val.shape, df_test.shape)

(27717, 246) (3080, 246) (3079, 246)


In [12]:
X_old_train, y_old_train = preprocess_sequence_data(df_train, target_column=target_column, seq_len=seq_len)

In [13]:
print(X_old_train.shape, y_old_train.shape)

(27589, 128, 246) (27589,)


In [14]:
new_df = df_cal.copy()
del new_df['Timestamp']

In [50]:
train_data, test_data = train_test_split(new_df, random_state=0, train_size=0.9, shuffle=False)

In [51]:
train_data, val_data = train_test_split(train_data, random_state=0, train_size=0.9, shuffle=False)

In [52]:
print(train_data.shape, val_data.shape, test_data.shape)

(24944, 246) (2772, 246) (3080, 246)


In [17]:
def preprocess_sequence_data(input_df, target_column='ZhangDieFu', seq_len=128):
    X_train, y_train = [], []
    df_value = input_df.values
    df_target_value = input_df[target_column].values
    for i in range(seq_len, len(df_value)):
        X_train.append(df_value[i-seq_len:i]) # Chunks of training data with a length of 128 df-rows
        y_train.append(df_target_value[i]) #Value of 4th column (Close Price) of df-row 128+1
    return np.array(X_train), np.array(y_train)

In [18]:
X_train, y_train = preprocess_sequence_data(train_data, target_column=target_column, seq_len=seq_len)

In [24]:
train_data[-3:]

,Open,High,Low,Close,Volume,open_MA_10,High_MA_10,Low_MA_10,Close_MA_10,Volume_MA_10,open_MA_ptc_10,High_MA_ptc_10,Low_MA_ptc_10,Close_MA_ptc_10,Volume_MA_ptc_10,Bias,Cmo_10,Atr_10,Cci_10,Volume_pct,ZhangDieFu,ZhenFu,Rsi_10,Ic_10,Macd,Signal,Histogram,UpperBB_10,MiddleBB_10,LowerBB_10,SlowK,SlowD,UpperMAE_10,MiddleMAE_10,LowerMAE_10,OBV,open_MA_20,High_MA_20,Low_MA_20,Close_MA_20,Volume_MA_20,open_MA_ptc_20,High_MA_ptc_20,Low_MA_ptc_20,Close_MA_ptc_20,Volume_MA_ptc_20,Cmo_20,Atr_20,Cci_20,Rsi_20,Ic_20,UpperBB_20,MiddleBB_20,LowerBB_20,UpperMAE_20,MiddleMAE_20,LowerMAE_20,open_MA_40,High_MA_40,Low_MA_40,Close_MA_40,Volume_MA_40,open_MA_ptc_40,High_MA_ptc_40,Low_MA_ptc_40,Close_MA_ptc_40,Volume_MA_ptc_40,Cmo_40,Atr_40,Cci_40,Rsi_40,Ic_40,UpperBB_40,MiddleBB_40,LowerBB_40,UpperMAE_40,MiddleMAE_40,LowerMAE_40,open_MA_60,High_MA_60,Low_MA_60,Close_MA_60,Volume_MA_60,open_MA_ptc_60,High_MA_ptc_60,Low_MA_ptc_60,Close_MA_ptc_60,Volume_MA_ptc_60,Cmo_60,Atr_60,Cci_60,Rsi_60,Ic_60,UpperBB_60,MiddleBB_60,LowerBB_60,UpperMAE_60,MiddleMAE_60,LowerMAE_60,open_MA_80,High_MA_80,Low_MA_80,Close_MA_80,Volume_MA_80,open_MA_ptc_80,High_MA_ptc_80,Low_MA_ptc_80,Close_MA_ptc_80,Volume_MA_ptc_80,Cmo_80,Atr_80,Cci_80,Rsi_80,Ic_80,UpperBB_80,MiddleBB_80,LowerBB_80,UpperMAE_80,MiddleMAE_80,LowerMAE_80,open_MA_100,High_MA_100,Low_MA_100,Close_MA_100,Volume_MA_100,open_MA_ptc_100,High_MA_ptc_100,Low_MA_ptc_100,Close_MA_ptc_100,Volume_MA_ptc_100,Cmo_100,Atr_100,Cci_100,Rsi_100,Ic_100,UpperBB_100,MiddleBB_100,LowerBB_100,UpperMAE_100,MiddleMAE_100,LowerMAE_100,open_MA_120,High_MA_120,Low_MA_120,Close_MA_120,Volume_MA_120,open_MA_ptc_120,High_MA_ptc_120,Low_MA_ptc_120,Close_MA_ptc_120,Volume_MA_ptc_120,Cmo_120,Atr_120,Cci_120,Rsi_120,Ic_120,UpperBB_120,MiddleBB_120,LowerBB_120,UpperMAE_120,MiddleMAE_120,LowerMAE_120,open_MA_140,High_MA_140,Low_MA_140,Close_MA_140,Volume_MA_140,open_MA_ptc_140,High_MA_ptc_140,Low_MA_ptc_140,Close_MA_ptc_140,Volume_MA_ptc_140,Cmo_140,Atr_140,Cci_140,Rsi_140,Ic_140,UpperBB_140,MiddleBB_140,LowerBB_140,UpperMAE_140,MiddleMAE_140,LowerMAE_140,open_MA_160,High_MA_160,Low_MA_160,Close_MA_160,Volume_MA_160,open_MA_ptc_160,High_MA_ptc_160,Low_MA_ptc_160,Close_MA_ptc_160,Volume_MA_ptc_160,Cmo_160,Atr_160,Cci_160,Rsi_160,Ic_160,UpperBB_160,MiddleBB_160,LowerBB_160,UpperMAE_160,MiddleMAE_160,LowerMAE_160,open_MA_180,High_MA_180,Low_MA_180,Close_MA_180,Volume_MA_180,open_MA_ptc_180,High_MA_ptc_180,Low_MA_ptc_180,Close_MA_ptc_180,Volume_MA_ptc_180,Cmo_180,Atr_180,Cci_180,Rsi_180,Ic_180,UpperBB_180,MiddleBB_180,LowerBB_180,UpperMAE_180,MiddleMAE_180,LowerMAE_180,open_MA_200,High_MA_200,Low_MA_200,Close_MA_200,Volume_MA_200,open_MA_ptc_200,High_MA_ptc_200,Low_MA_ptc_200,Close_MA_ptc_200,Volume_MA_ptc_200,Cmo_200,Atr_200,Cci_200,Rsi_200,Ic_200,UpperBB_200,MiddleBB_200,LowerBB_200,UpperMAE_200,MiddleMAE_200,LowerMAE_200
25141,22688.9,22860.0,22662.8,22774.4,29310.59,23345.63,23538.72,23124.89,23271.99,36557.3301,-0.004228,-0.003707,-0.003264,-0.003155,-0.038090,6.766175,-30.531791,388.080898,-66.666667,-0.359439,0.003764,0.008691,34.734104,-0.018268,-65.956234,75.151729,-141.107963,23928.611251,23271.99,22615.368749,10.702178,19.141492,23928.611251,23271.99,22615.368749,-1963003.041,23535.27,23731.410,23345.225,23505.140,41840.77235,-0.002304,-0.002105,-0.001502,-0.00128,-0.012807,-11.702502,381.461939,-66.666667,44.148749,-0.023866,24250.866659,23505.140,22759.413341,24250.866659,23505.140,22759.413341,23347.5025,23559.3075,23169.3650,23367.6125,44221.439125,0.000741,0.000661,0.000824,0.000864,-0.006470,0.992301,356.675036,-66.666667,50.496150,-0.017672,24190.274629,23367.6125,22544.950371,24190.274629,23367.6125,22544.950371,22883.801667,23086.223333,22699.453333,22885.455000,42094.086200,0.000352,0.000043,0.000376,0.000073,-0.011779,4.903716,333.802641,-66.666667,52.451858,-0.006968,24427.374107,22885.455000,21343.535893,24427.374107,22885.455000,21343.535893,22564.94875,22765.37375,22400.07750,22586.83625,40279.961637,0.000955,0.000980,0.0

In [19]:
X_train.shape, y_train.shape

((24816, 128, 246), (24816,))

In [39]:
X_train, y_train = [], []
df_value = train_data.values
df_target_value = train_data[target_column].values
for i in range(seq_len, len(df_value)):
    X_train.append(df_value[i-seq_len:i]) # Chunks of training data with a length of 128 df-rows
    y_train.append(df_target_value[i]) #Value of 4th column (Close Price) of df-row 128+1


In [48]:
train_data[-129:]

,Open,High,Low,Close,Volume,open_MA_10,High_MA_10,Low_MA_10,Close_MA_10,Volume_MA_10,open_MA_ptc_10,High_MA_ptc_10,Low_MA_ptc_10,Close_MA_ptc_10,Volume_MA_ptc_10,Bias,Cmo_10,Atr_10,Cci_10,Volume_pct,ZhangDieFu,ZhenFu,Rsi_10,Ic_10,Macd,Signal,Histogram,UpperBB_10,MiddleBB_10,LowerBB_10,SlowK,SlowD,UpperMAE_10,MiddleMAE_10,LowerMAE_10,OBV,open_MA_20,High_MA_20,Low_MA_20,Close_MA_20,Volume_MA_20,open_MA_ptc_20,High_MA_ptc_20,Low_MA_ptc_20,Close_MA_ptc_20,Volume_MA_ptc_20,Cmo_20,Atr_20,Cci_20,Rsi_20,Ic_20,UpperBB_20,MiddleBB_20,LowerBB_20,UpperMAE_20,MiddleMAE_20,LowerMAE_20,open_MA_40,High_MA_40,Low_MA_40,Close_MA_40,Volume_MA_40,open_MA_ptc_40,High_MA_ptc_40,Low_MA_ptc_40,Close_MA_ptc_40,Volume_MA_ptc_40,Cmo_40,Atr_40,Cci_40,Rsi_40,Ic_40,UpperBB_40,MiddleBB_40,LowerBB_40,UpperMAE_40,MiddleMAE_40,LowerMAE_40,open_MA_60,High_MA_60,Low_MA_60,Close_MA_60,Volume_MA_60,open_MA_ptc_60,High_MA_ptc_60,Low_MA_ptc_60,Close_MA_ptc_60,Volume_MA_ptc_60,Cmo_60,Atr_60,Cci_60,Rsi_60,Ic_60,UpperBB_60,MiddleBB_60,LowerBB_60,UpperMAE_60,MiddleMAE_60,LowerMAE_60,open_MA_80,High_MA_80,Low_MA_80,Close_MA_80,Volume_MA_80,open_MA_ptc_80,High_MA_ptc_80,Low_MA_ptc_80,Close_MA_ptc_80,Volume_MA_ptc_80,Cmo_80,Atr_80,Cci_80,Rsi_80,Ic_80,UpperBB_80,MiddleBB_80,LowerBB_80,UpperMAE_80,MiddleMAE_80,LowerMAE_80,open_MA_100,High_MA_100,Low_MA_100,Close_MA_100,Volume_MA_100,open_MA_ptc_100,High_MA_ptc_100,Low_MA_ptc_100,Close_MA_ptc_100,Volume_MA_ptc_100,Cmo_100,Atr_100,Cci_100,Rsi_100,Ic_100,UpperBB_100,MiddleBB_100,LowerBB_100,UpperMAE_100,MiddleMAE_100,LowerMAE_100,open_MA_120,High_MA_120,Low_MA_120,Close_MA_120,Volume_MA_120,open_MA_ptc_120,High_MA_ptc_120,Low_MA_ptc_120,Close_MA_ptc_120,Volume_MA_ptc_120,Cmo_120,Atr_120,Cci_120,Rsi_120,Ic_120,UpperBB_120,MiddleBB_120,LowerBB_120,UpperMAE_120,MiddleMAE_120,LowerMAE_120,open_MA_140,High_MA_140,Low_MA_140,Close_MA_140,Volume_MA_140,open_MA_ptc_140,High_MA_ptc_140,Low_MA_ptc_140,Close_MA_ptc_140,Volume_MA_ptc_140,Cmo_140,Atr_140,Cci_140,Rsi_140,Ic_140,UpperBB_140,MiddleBB_140,LowerBB_140,UpperMAE_140,MiddleMAE_140,LowerMAE_140,open_MA_160,High_MA_160,Low_MA_160,Close_MA_160,Volume_MA_160,open_MA_ptc_160,High_MA_ptc_160,Low_MA_ptc_160,Close_MA_ptc_160,Volume_MA_ptc_160,Cmo_160,Atr_160,Cci_160,Rsi_160,Ic_160,UpperBB_160,MiddleBB_160,LowerBB_160,UpperMAE_160,MiddleMAE_160,LowerMAE_160,open_MA_180,High_MA_180,Low_MA_180,Close_MA_180,Volume_MA_180,open_MA_ptc_180,High_MA_ptc_180,Low_MA_ptc_180,Close_MA_ptc_180,Volume_MA_ptc_180,Cmo_180,Atr_180,Cci_180,Rsi_180,Ic_180,UpperBB_180,MiddleBB_180,LowerBB_180,UpperMAE_180,MiddleMAE_180,LowerMAE_180,open_MA_200,High_MA_200,Low_MA_200,Close_MA_200,Volume_MA_200,open_MA_ptc_200,High_MA_ptc_200,Low_MA_ptc_200,Close_MA_ptc_200,Volume_MA_ptc_200,Cmo_200,Atr_200,Cci_200,Rsi_200,Ic_200,UpperBB_200,MiddleBB_200,LowerBB_200,UpperMAE_200,MiddleMAE_200,LowerMAE_200
25015,20923.3,21009.1,20812.4,20956.5,15281.640,20888.75,21023.71,20790.45,20907.97,29297.9820,0.000307,0.000334,0.000294,0.000921,-0.027566,1.288784,14.349529,231.632797,66.666667,-0.491846,0.001592,0.009401,57.174764,0.002330,154.756550,162.300733,-7.544184,21149.379943,20907.97,20666.560057,64.394351,72.928035,21149.379943,20907.97,20666.560057,-2192898.987,20791.625,20923.490,20707.000,20812.290,26691.95320,0.000730,0.000440,0.000924,0.001144,-0.056312,16.329939,237.037973,66.666667,58.164969,0.007445,21126.170524,20812.290,20498.409476,21126.170524,20812.290,20498.409476,20503.6200,20641.9075,20401.9275,20525.5100,30424.081600,0.000997,0.001057,0.001127,0.001144,-0.009500,12.303931,238.139676,66.666667,56.151966,0.017305,21335.051886,20525.5100,19715.968114,21335.051886,20525.5100,19715.968114,20283.608333,20436.343333,20164.980000,20302.215000,35651.783783,0.000926,0.000982,0.000924,0.000968,-0.001339,8.723448,235.231546,66.666667,54.361724,0.023280,21291.266867,20302.215000,19313.163133,21291.266867,20302.215000,19313.163133,20109.98500,20247.72875,19999.38875,20122.33625,32539.515137,0.000713,0.000666,0.000708,0.000652,

In [43]:
df_value[i-seq_len:i]

array([[20923.3       , 21009.1       , 20812.4       , ...,
        22249.8223256 , 20689.8525    , 19129.8826744 ],
       [20956.6       , 20980.        , 20804.1       , ...,
        22249.80445306, 20689.8405    , 19129.87654694],
       [20879.5       , 20936.3       , 20755.        , ...,
        22247.60534619, 20688.531     , 19129.45665381],
       ...,
       [23228.9       , 23330.        , 22684.        , ...,
        23860.92993229, 21314.476     , 18768.02206771],
       [22688.9       , 22860.        , 22662.8       , ...,
        23872.04264523, 21331.1005    , 18790.15835477],
       [22774.4       , 22922.        , 22636.        , ...,
        23883.22923517, 21348.7865    , 18814.34376483]])

In [47]:
df_target_value[i]

-0.0030852948369513022

In [34]:
y_train

[-0.001038633871634942]

In [ ]:
y_train